# Shopping Time Optimization - YAP441
Zeynep Meriç Aşık - 201410026

In [1]:
#!pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np

## Market Design

In [81]:
class Product:
    def __init__(self, name, wait_time=1, price=1):
        self.name = name
        self.wait_time = wait_time  # in minutes
        self.price = price

    def __str__(self):
        return f"{self.name} (Waiting Time: {self.wait_time} minutes, Price: {self.price} TL)"

In [82]:
class Graph:
    def __init__(self):
        self.adjacency_list = {}

        
    def add_vertex(self, vertex):
        if vertex not in self.adjacency_list:
            self.adjacency_list[vertex] = []

            
    def add_edge(self, from_vertex, to_vertex, weight):
        if from_vertex in self.adjacency_list and to_vertex in self.adjacency_list:
            self.adjacency_list[from_vertex].append((to_vertex, weight))
            self.adjacency_list[to_vertex].append((from_vertex, weight))

            
    def get_path_weight(self, path): # get the total weights through the path
        total_weight = 0
        for i in range(len(path) - 1):
            for neighbor in self.adjacency_list[path[i]]:
                if neighbor[0] == path[i + 1]:
                    total_weight += neighbor[1]
                    break
        return total_weight
    
    
    def get_wait_time(self, path, product_wait_times):  # get the total wait time through the path
        total_weight = 0
        for i in range(len(path) - 1):
            if path[i] in product_wait_times:
                total_weight += product_wait_times[path[i]]

        if path[-1] in product_wait_times:
            total_weight += product_wait_times[path[-1]]

        return total_weight

    
    def display_products(self):
        corridor_to_products = {}

        for key, neighbors in self.adjacency_list.items():
            if not isinstance(key, Product):# if it is a corridor
                corridor_to_products[key] = []# add corridor to dictionary

                for neighbor in neighbors:
                    if isinstance(neighbor[0], Product):# if it is a product and a neighbor
                        corridor_to_products[key].append(neighbor[0].name)# add it to this corridor's product list

        del corridor_to_products["Checkout"]# checkout is not a corridor
        
        for corridor, products in corridor_to_products.items():
            print(f"{corridor}: {products}")
    
    
    def display_connected_corridors(self):
        corridor_connections = {}

        for key, neighbors in self.adjacency_list.items():
            if not isinstance(key, Product):# if it is a corridor
                corridor_connections[key] = []# initialize corridor

                for neighbor in neighbors:
                    if not isinstance(neighbor[0], Product):# if it is a neighbor but not a product
                        corridor_connections[key].append(neighbor[0])# it is a connected corridor

        for corridor, connections in corridor_connections.items():
            print(f"{corridor}: {connections}")

In [83]:
# Product List

# Fruits and Vegetables
apple = Product("Apple", price=5)
banana = Product("Banana", price=4)
orange = Product("Orange", price=6)
grapes = Product("Grapes", price=8)
strawberry = Product("Strawberry", price=10)
pear = Product("Pear", price=5)
kiwi = Product("Kiwi", price=7)
carrot = Product("Carrot", price=3)
potato = Product("Potato", price=2)
tomato = Product("Tomato", price=4)
lettuce = Product("Lettuce", price=3)
cucumber = Product("Cucumber", price=2)
peppers = Product("Peppers", price=5)
onion = Product("Onion", price=2)
mushrooms = Product("Mushrooms", wait_time=2, price=6)

# Dairy
milk = Product("Milk", price=4)
cheese = Product("Cheese", wait_time=2, price=30)
yoghurt = Product("Yogurt", wait_time=1, price=9)
butter = Product("Butter", price=15)
cream = Product("Cream", price=7)
eggs = Product("Eggs", wait_time=1, price=10)

# Bakery
bread = Product("Bread", price=3)
croissant = Product("Croissant", price=5)
baguette = Product("Baguette", price=4)
muffin = Product("Muffin", price=2)
cake = Product("Cake", wait_time=5, price=20)
pie = Product("Pie", wait_time=4, price=15)

# Meat and Fish
meat = Product("Meat", wait_time=5, price=50)
fish = Product("Fish", wait_time=10, price=40)
chicken = Product("Chicken", wait_time=3, price=25)
sausage = Product("Sausage", price=20)
bacon = Product("Bacon", price=22)
shrimp = Product("Shrimp", wait_time=6, price=35)

# Beverages
orange_juice = Product("Orange Juice", price=6)
coffee = Product("Coffee", wait_time=2, price=15)
tea = Product("Tea", price=12)
water = Product("Water", price=1)
soda = Product("Soda", price=3)
beer = Product("Beer", price=5)
wine = Product("Wine", price=15)

# Snacks and Others
chocolate = Product("Chocolate", price=7)
chips = Product("Chips", price=4)
nuts = Product("Nuts", price=10)
crackers = Product("Crackers", price=5)
pasta = Product("Pasta", price=8)
rice = Product("Rice", price=5)
cereal = Product("Cereal", price=8)
honey = Product("Honey", wait_time=2, price=12)

In [84]:
market_graph = Graph()

# adding corridors
market_graph.add_vertex("Bakery")
market_graph.add_vertex("Meat and Fish")
market_graph.add_vertex("Fruits and Vegetables")
market_graph.add_vertex("Dairy")
market_graph.add_vertex("Beverages")
market_graph.add_vertex("Snacks")
market_graph.add_vertex("Checkout")

# connecting corridors
market_graph.add_edge("Bakery", "Snacks", 2)
market_graph.add_edge("Bakery", "Beverages", 2)
market_graph.add_edge("Meat and Fish", "Dairy", 2)
market_graph.add_edge("Meat and Fish", "Fruits and Vegetables", 2)
market_graph.add_edge("Snacks", "Beverages", 2)
market_graph.add_edge("Snacks", "Checkout", 3)
market_graph.add_edge("Beverages", "Dairy", 2)
market_graph.add_edge("Beverages", "Checkout", 3)
market_graph.add_edge("Dairy", "Fruits and Vegetables", 2)
market_graph.add_edge("Dairy", "Checkout", 3)
market_graph.add_edge("Fruits and Vegetables", "Checkout", 3)

# adding products to their corridors
product_corridor_map = {
    "Bakery": [bread, croissant, baguette, muffin, cake, pie],
    "Meat and Fish": [meat, fish, chicken, sausage, bacon, shrimp],
    "Fruits and Vegetables": [apple, banana, orange, grapes, strawberry, pear, kiwi, carrot, potato, tomato, lettuce, cucumber, peppers, onion, mushrooms],
    "Dairy": [milk, cheese, yoghurt, butter, cream, eggs],
    "Beverages": [orange_juice, coffee, tea, water, soda, beer, wine],
    "Snacks": [chocolate, chips, nuts, crackers, pasta, rice, cereal, honey]
}

# adding products to the graph
for corridor, products in product_corridor_map.items():
    for product in products:
        market_graph.add_vertex(product)
        market_graph.add_edge(corridor, product, 1)

# connect the products within each corridor
for products in product_corridor_map.values():
    for i in range(len(products) - 1):
        # connecting each product to the next product within the same corridor
        market_graph.add_edge(products[i], products[i+1], 1)

In [85]:
# display which product is in which corridor
market_graph.display_products()

Bakery: ['Bread', 'Croissant', 'Baguette', 'Muffin', 'Cake', 'Pie']
Meat and Fish: ['Meat', 'Fish', 'Chicken', 'Sausage', 'Bacon', 'Shrimp']
Fruits and Vegetables: ['Apple', 'Banana', 'Orange', 'Grapes', 'Strawberry', 'Pear', 'Kiwi', 'Carrot', 'Potato', 'Tomato', 'Lettuce', 'Cucumber', 'Peppers', 'Onion', 'Mushrooms']
Dairy: ['Milk', 'Cheese', 'Yogurt', 'Butter', 'Cream', 'Eggs']
Beverages: ['Orange Juice', 'Coffee', 'Tea', 'Water', 'Soda', 'Beer', 'Wine']
Snacks: ['Chocolate', 'Chips', 'Nuts', 'Crackers', 'Pasta', 'Rice', 'Cereal', 'Honey']


In [86]:
# display the connected corridors
market_graph.display_connected_corridors()

Bakery: ['Snacks', 'Beverages']
Meat and Fish: ['Dairy', 'Fruits and Vegetables']
Fruits and Vegetables: ['Meat and Fish', 'Dairy', 'Checkout']
Dairy: ['Meat and Fish', 'Beverages', 'Fruits and Vegetables', 'Checkout']
Beverages: ['Bakery', 'Snacks', 'Dairy', 'Checkout']
Snacks: ['Bakery', 'Beverages', 'Checkout']
Checkout: ['Snacks', 'Beverages', 'Dairy', 'Fruits and Vegetables']


In [41]:
# usage example          ######del
print(apple)

product_wait_times = {
    apple: apple.wait_time,
    meat: meat.wait_time,
    fish: fish.wait_time
}

path = [apple, meat, fish, "Checkout"]
total_time = market_graph.get_wait_time(path, product_wait_times)
print(f"Total time for the path including wait times: {total_time} minutes")

Apple (Waiting Time: 1 minutes, Price: 5 TL)
Total time for the path including wait times: 16 minutes


In [52]:
#############3
isinstance(apple, Product)

True

## Algorithm to optimize shopping time